In [1]:
import tkinter as tk
from tkinter import messagebox
from queue import Queue

class LibraryManagementSystem:
    def __init__(self):
        self.books = []  # List of books
        self.request_queue = Queue()  # Request queue for FCFS scheduling

    def add_book(self, book_name, author, quantity, isbn):
        self.books.append({
            "book_name": book_name,
            "author": author,
            "quantity": quantity,
            "isbn": isbn,
            "issued": False
        })
        messagebox.showinfo("Success", f"Book '{book_name}' added successfully!")

    def display_books(self):
        if not self.books:
            return "No books available!"
        result = ""
        for book in self.books:
            result += (f"Name: {book['book_name']}, Author: {book['author']}, "
                       f"Quantity: {book['quantity']}, ISBN: {book['isbn']}, "
                       f"Issued: {'Yes' if book['issued'] else 'No'}\n")
        return result

    def request_books(self, user_name, requests):
        for request in requests:
            isbn, quantity = request
            # Check if the book exists and has enough copies
            for book in self.books:
                if book["isbn"] == isbn:
                    if book["quantity"] >= quantity:
                        # Add request to the FCFS queue
                        self.request_queue.put({"user_name": user_name, "isbn": isbn, "quantity": quantity})
                    else:
                        messagebox.showwarning(
                            "Error", f"Not enough copies available for ISBN {isbn}!")
                    break
            else:
                messagebox.showwarning("Error", f"Book with ISBN {isbn} not found!")
        messagebox.showinfo("Success", "All valid requests have been added!")

    def process_requests(self):
        if self.request_queue.empty():
            messagebox.showinfo("Info", "No requests to process!")
            return

        result = ""
        while not self.request_queue.empty():
            request = self.request_queue.get()
            for book in self.books:
                if book["isbn"] == request["isbn"]:
                    book["quantity"] -= request["quantity"]
                    if book["quantity"] == 0:
                        book["issued"] = True
                    result += (f"Request Processed: {request['user_name']} borrowed "
                               f"{request['quantity']} copies of '{book['book_name']}'\n")
                    break
        messagebox.showinfo("Requests Processed", result)

# GUI
class LibraryGUI:
    def __init__(self, root, system):
        self.system = system
        self.root = root
        self.root.title("Library Management System")
        self.root.geometry("600x500")

        # Add Book Section
        self.add_book_frame = tk.LabelFrame(root, text="Add Book", padx=10, pady=10)
        self.add_book_frame.pack(fill="both", expand="yes", padx=10, pady=5)

        tk.Label(self.add_book_frame, text="Book Name:").grid(row=0, column=0)
        tk.Label(self.add_book_frame, text="Author:").grid(row=1, column=0)
        tk.Label(self.add_book_frame, text="Quantity:").grid(row=2, column=0)
        tk.Label(self.add_book_frame, text="ISBN:").grid(row=3, column=0)

        self.book_name_entry = tk.Entry(self.add_book_frame)
        self.author_entry = tk.Entry(self.add_book_frame)
        self.quantity_entry = tk.Entry(self.add_book_frame)
        self.isbn_entry = tk.Entry(self.add_book_frame)

        self.book_name_entry.grid(row=0, column=1)
        self.author_entry.grid(row=1, column=1)
        self.quantity_entry.grid(row=2, column=1)
        self.isbn_entry.grid(row=3, column=1)

        tk.Button(self.add_book_frame, text="Add Book", command=self.add_book).grid(row=4, column=0, columnspan=2)

        # Display Books Section
        self.display_frame = tk.LabelFrame(root, text="Library Books", padx=10, pady=10)
        self.display_frame.pack(fill="both", expand="yes", padx=10, pady=5)

        self.books_text = tk.Text(self.display_frame, height=10, width=50)
        self.books_text.pack()
        tk.Button(self.display_frame, text="Display Books", command=self.display_books).pack()

        # Request Book Section
        self.request_frame = tk.LabelFrame(root, text="Request Books (Multiple)", padx=10, pady=10)
        self.request_frame.pack(fill="both", expand="yes", padx=10, pady=5)

        tk.Label(self.request_frame, text="User Name:").grid(row=0, column=0)
        tk.Label(self.request_frame, text="Requests (ISBN:Quantity, e.g., 101:2):").grid(row=1, column=0)

        self.user_name_entry = tk.Entry(self.request_frame)
        self.requests_entry = tk.Entry(self.request_frame)

        self.user_name_entry.grid(row=0, column=1)
        self.requests_entry.grid(row=1, column=1)

        tk.Button(self.request_frame, text="Request Books", command=self.request_books).grid(row=2, column=0, columnspan=2)

        # Process Requests Button
        tk.Button(root, text="Process Requests", command=self.process_requests).pack(pady=10)

    def add_book(self):
        book_name = self.book_name_entry.get()
        author = self.author_entry.get()
        quantity = int(self.quantity_entry.get())
        isbn = int(self.isbn_entry.get())
        self.system.add_book(book_name, author, quantity, isbn)

    def display_books(self):
        books_info = self.system.display_books()
        self.books_text.delete("1.0", tk.END)
        self.books_text.insert(tk.END, books_info)

    def request_books(self):
        user_name = self.user_name_entry.get()
        requests_raw = self.requests_entry.get()
        requests = []
        try:
            for req in requests_raw.split(","):
                isbn, quantity = map(int, req.split(":"))
                requests.append((isbn, quantity))
            self.system.request_books(user_name, requests)
        except ValueError:
            messagebox.showerror("Error", "Invalid request format! Use ISBN:Quantity, e.g., 101:2,102:1")

    def process_requests(self):
        self.system.process_requests()

# Main
if __name__ == "__main__":
    library_system = LibraryManagementSystem()
    root = tk.Tk()
    gui = LibraryGUI(root, library_system)
    root.mainloop()
